# WORD PREDICTION IN N-GRAM LANGUAGE MODELLING USING STUPID BACKOFF

### ABSTRACT

In order to predict the next word, N-gram language modelling technique is used with a smoothing method called _Stupid Backoff_ to calculate the relative frequencies of the word in n-gram and generate a score to identify the most likely next word. The model was evaluated using the concept of information theory by calculating the _shannon entropy_ and _perplexity_ to understand how certain/uncertain the model was while predicting the word which for our case gave average entropy 3 and average perplexity of just 31 which means the model was just uncertain for 31 alternative predictions.

### INTRODUCTION

Stupid Back-off model takes the maximum-likelihood estimator of each potential completed n-gram as the ratio between the number of occurrences of that n-gram in the training set and the number occurrences of the (n-1)-word “prefix” for this n-gram.  If this ratio is zero (i.e. the n-gram is not in the training set), the model “backs off” to look at the ratio between the n-gram and the (n-2)-word prefix obtained by lopping the first word from the original prefix. This process continues recursively, multiplying by a factor between 0 and 1 (say 'k') each time. It calculates this k parameter (heuristically set at a constant 0.4 in practice) to generate scores to identify the next most likely word. These scores are than renormalized to add up to 1 for all potential words that complete an n-gram and is listed in descending order to get the most likely word. Hence, there is no need to actually calculate the true probability of the next word as it is done in Katz back-off model. It can be evaluated on examining the perplexity and the concept of shannon extropy to see how well the prediction is which is based on the concept of information theory.

### Importing libraries:

In [1]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from io import StringIO

### Data loading and Text Pre-processing:

In [0]:
# Reading N lines of data from input file
N = 10000
with open("ptb.txt") as file:
    lines = [next(file) for x in range(N)]
joined_lines = [" ".join(lines)]

In [0]:
def clean_article(lines):
  
  '''
  Returns a cleaned corpus of text after text preprocessing
  
  Input: lines which are present in the text corpus
  
  Output: Cleaned text without letters, replacing it with space and single letter pronoun like "I" or article like "a".
  
  '''
  art1 = re.sub("[^A-Za-z]", ' ', lines)          # substitutes a space for characters which are not in A-Za-z
  art2 = re.sub("\s[B-HJ-Zb-hj-z]\s", ' ', art1)  # substitutes a space for all characters EXCEPT A and I
  art3 = re.sub("^[B-HJ-Zb-hj-z]\s", ' ', art2)   # substitutes a space for characters which does not begin with A-Za-z EXCEPT A and I
  art4 = re.sub("\s[B-HJ-Zb-hj-z]$", ' ', art3)   # substitute a space for characters which ends with A-Za-z EXCEPT A and I
  return art4.lower()                             # converts to lowercase

In [0]:
# Tokenizes the words on whitespace with n-gram of length 1 to 5 and puts their counts into a Pandas dataframe with the n-grams as column names.
# Converts a collection of text to a matrix of token counts
ngram_bow = CountVectorizer(stop_words = None, preprocessor = clean_article, tokenizer = WhitespaceTokenizer().tokenize, ngram_range=(1,5), 
                            max_features = None, max_df = 1.0, min_df = 1, binary = False)

# Fitting and transforming the model to create a sparse matrix of token counts
ngram_count_sparse = ngram_bow.fit_transform(joined_lines)

In [5]:
# Converting the matrix to pandas dataframe
ngram_count = pd.DataFrame(ngram_count_sparse.toarray())
ngram_count.T.head()

,0
0,5257
1,4
2,1
3,1
4,1


In [6]:
# Creating column with feature names in n-gram
ngram_count.columns = ngram_bow.get_feature_names()
ngram_count.T.head(10)

,0
a,5257
a a,4
a a diversified,1
a a diversified construction,1
a a diversified construction concern,1
a a french,1
a a french state,1
a a french state controlled,1
a a share,1
a a share offer,1


In [7]:
# decriptive statistics
ngram_count.T.describe()

,0
count,661747.000000
mean,1.533743
std,26.465793
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,12293.000000


**Observation:**

1. The minimum count of word in the n-gram corpus is 1 and maximum being 12293
2. Total n-gram count of word is 6,61,747 in the text corpus
3. Average n-gram count is around 1.5 and each n-gram word is aproximately 26 standard deviations away from each other

In [8]:
# Find the top 10 most used words in the n-gram having count greater than 0
ngram_sums = ngram_count.sum(axis = 0)
ngram_sums = ngram_sums[ngram_sums > 0]
ngram_sums.sort_values(axis=0, ascending=False).head(10)

the     12293
unk     10913
of       5871
to       5610
a        5257
in       4401
and      4118
for      2157
that     2157
is       1760
dtype: int64

In [0]:
# Convert n-gram count dataframe into a Pandas series with the n-grams as indices for ease of working with the counts.  
ngrams = list(ngram_sums.index.values)

### Stupid Back-off Language Modelling in N-grams:

In [0]:
def count_of_onegrams(ngram_sums):
  '''
  Returns count of n-gram occurances as Integer
  
  Input: Words present in n-gram
  
  Output: Total number of occurances of 1-grams to calculate 1-gram frequency
  '''
  onegrams = 0
  for ng in ngrams:
    ng_split = ng.split(" ")
    if len(ng_split) == 1:
      onegrams += ngram_sums[ng]
  return onegrams

In [0]:
# This is the last resort of the back-off algorithm if the n-gram completion does not occur in the corpus with any of the prefix words.
def base_freq(og):
    '''
    Returns frequencies of the 1-gram as Series
  
    Input: Words present in 1-gram
  
    Output: Series of 1-gram frequencies
  '''
    freqs = pd.Series()
    for ng in ngrams:
        ng_split = ng.split(" ")
        if len(ng_split) == 1:
            freqs[ng] = ngram_sums[ng] / og
    return freqs

In [0]:
# Calculating base freq for all onegrams
bf = base_freq(count_of_onegrams(ngram_sums))

In [0]:
def calculate_scores(prefix, chops, factor = 0.4):
    '''
    The function below finds any n-grams that are completions of a given prefix phrase with a specified number (could be zero) of words 'chopped' off the beginning.  
    
    For each, it calculates the count ratio of the completion to the (chopped) prefix, tabulating them in a series to be returned by the function.  

    If the number of chops equals the number of words in the prefix (i.e. all prefix words are chopped), the 1-gram base frequencies are returned.
  
    '''
    cs = pd.Series()
    prefix_split = prefix.split(" ")
    l = len(prefix_split)
    prefix_split_chopped = prefix_split[chops:l]
    new_l = l - chops
    if new_l == 0:
        return factor**chops * bf
    prefix_chopped = ' '.join(prefix_split_chopped)
    for ng in ngrams:
        ng_split = ng.split(" ")
        if (len(ng_split) == new_l + 1) and (ng_split[0:new_l] == prefix_split_chopped):
            cs[ng_split[-1]] = factor**chops * ngram_sums[ng] / ngram_sums[prefix_chopped]
    return cs

In [14]:
# Example of completion scores:
calculate_scores('I am going', 0, 0.4)

Series([], dtype: float64)

In [0]:
def score_given(input, fact):
    '''
    The function takes different numbers of 'chops' up to the length of the prefix to output a combined list of scores for potential completion of input
  
    '''
    sg = pd.Series()
    given_split = input.split(" ")
    given_length = len(given_split)
    for i in range(given_length+1):
        fcs = calculate_scores(input, i, fact)
        for i in fcs.index:
            if i not in sg.index:
                sg[i] = fcs[i]
    return sg

In [0]:
def score_output(input, fact):
    '''
    This function takes potential completion scores as input, sorts in descending order and re-normalizes them as a percentage (pseudo-probability)
    
    '''
    sg = score_given(input, fact)
    ss = sg.sum()
    sg = 100 * sg / ss
    sg.sort_values(axis=0, ascending=False, inplace=True)
    return round(sg,1)

In [17]:
# sample next word prediction scores
score_output('I am going', 0.4)[0:15]

to         51.3
back        4.3
into        3.2
unk         2.1
the         1.8
against     1.1
down        1.1
sour        1.1
over        1.1
on          1.1
much        1.1
like        1.1
for         1.1
through     1.1
along       1.1
dtype: float64

### MODEL EVALUATION:

Using the concept of information theory, we can not only find out how well the model does with particular test prefixes (comparing predictions to actual completions), but also how uncertain it is given particular test prefixes (i.e. unlabeled data).

Let us think about the case where the model predicts all of the training 1-grams (let’s say there is M of them) with equal probability. hen, we could place all of the 1-grams in a binary tree, and then by asking log (base 2) of M questions of someone who knew the actual completion, we could find the correct prediction. This quantity (log base 2 of M) is known as **entropy (symbol H)** and in general is defined as H = - ∑ (p_i * log(p_i)) where i goes from 1 to M and p_i is the predicted probability score for 1-gram i. (If p_i is always 1/M, we have H = -∑((1/M) * log(1/M)) for i from 1 to M. This is just M * -((1/M) * log(1/M)), which simplifies to -log(1/M), which further simplifies to log(M).) Entropy is expressed in bits (if the log chosen is base 2) since it is the number of yes/no questions needed to identify a word. If some of the p_i values are higher than others, entropy goes down since we can structure the binary tree to place more common words in the top layers, thus finding them faster as we ask questions.

To encapsulate uncertainty of the model, we can use a metric called **perplexity**, which is 2^H, as calculated for a given test prefix. We can then take the average perplexity over the test prefixes to evaluate our model. In special case of equal probabilities assigned to each prediction, perplexity would be 2^log(M), i.e. just M. This means that perplexity is at most M, i.e. the model is “M-ways uncertain.” It can’t make a choice among M alternatives. If the probabilities are less uniformly distributed, entropy (H) and thus perplexity is lower.

In [18]:
# Read N lines from the test data set
N = 1000
with open("ptb.test.txt", "r+") as text:
  articles_test = [next(text) for x in range(N)]
  article = text.read().split()
joined_articles_test = [" ".join(articles_test)]
# Converts a collection of text to a matrix of token counts
count_vect = CountVectorizer(input="file")
docs_new = [ StringIO(x) for x in article ]
# Fitting and transforming the model to create a sparse matrix of token counts
X_train_counts = count_vect.fit(docs_new)
len(X_train_counts.vocabulary_)
article_transformed = X_train_counts.transform(docs_new)
# Get the shape of transformed article
print (article_transformed.shape)

(56909, 5210)


In [0]:
# Tokenize test words into n-grams of length 5

# Tokenizes the words on whitespace with n-gram of length 5 and puts their counts into a Pandas dataframe with the n-grams as column names.
# Converts a collection of text to a matrix of token counts
test_bow = CountVectorizer(stop_words = None, preprocessor = clean_article, tokenizer = WhitespaceTokenizer().tokenize, ngram_range=(5,5), 
                           max_features = None, max_df = 1.0, min_df = 1, binary = False)

# Fitting and transforming the model to create a sparse matrix of token counts
ngram_count_sparse_test = test_bow.fit_transform(joined_articles_test)

# Converting matrix to pandas dataframe
test_ngram_count = pd.DataFrame(ngram_count_sparse_test.toarray())

# Creating a column containing ngram words as feature vector
test_ngram_count.columns = test_bow.get_feature_names()

# Find most used words in ngram having count greater than 1
test_sums = test_ngram_count.sum(axis = 0)
test_sums = test_sums[test_sums > 1]

# Convert n-gram count dataframe into a Pandas series with the n-grams as indices for ease of working with the counts.  
test_ngrams = list(test_sums.index.values)

In [0]:
# The helper functions below give the number of occurrences of n-grams in order to explore and calculate frequencies
def number_of_unique_ngrams(n, ngrams):
    '''
    Returns count of unique n-gram occurances as Integer
  
    Input: Words present in n-gram
  
    Output: Total number of occurances of unique n-grams to calculate n-gram frequency
    '''
    grams = 0
    for ng in ngrams:
        ng_split = ng.split(" ")
        if len(ng_split) == n:
            grams += 1
    return grams

def number_of_ngrams(n, ngrams):
    '''
    Returns count of n-gram occurances as Integer
  
    Input: Words present in n-gram
  
    Output: Total number of occurances of n-grams to calculate n-gram frequency
    '''
    grams = 0
    for ng in ngrams:
        ng_split = ng.split(" ")
        if len(ng_split) == n:
            grams += ngram_sums[ng]
    return grams

# This is the last resort of the back-off algorithm if the n-gram completion does not occur in the corpus with any of the prefix words.
def base_freq(n, ngrams):
    '''
    Returns frequencies of the 1-gram as Series
  
    Input: Words present in 1-gram
  
    Output: Series of 1-gram frequencies
    '''
    tot_ngrams = number_of_ngrams(n, ngrams)
    freqs = pd.Series()
    for ng in ngrams:
        ng_split = ng.split(" ")
        if len(ng_split) == n:
            freqs[ng] = ngram_sums[ng] / tot_ngrams
    return freqs

In [0]:
# Calculating base freq for all ngrams so as not to re-calculate multiple times:
bf = base_freq(1, ngrams)

In [0]:
def score_output(input, fact):
    '''
    This function takes potential completion scores as input, sorts in descending order and re-normalizes them as a percentage (pseudo-probability)
    
    '''
    sg = score_given(input, fact)
    ss = sg.sum()
    sg = sg / ss
    sg.sort_values(axis=0, ascending=False, inplace=True)
    return sg

In [0]:
# Random generation of test samle
num_test_grams = len(test_ngrams)
import random
r = random.sample(range(num_test_grams), 75)
test_gram_sample = [test_ngrams[i] for i in r]

In [0]:
# create empty lists
entropies = []
top1 = []
top3 = []
top10 = []
ranks = []

for gram in test_gram_sample:
    words = gram.split(' ')
    ending_word = words[4]
    pref = ' '.join(words[0:4])
    so = score_output(pref, 0.4)
    # calculate the entropy
    entropy = sum(-1 * so * np.log2(so))
    entropies.append(entropy)
    # completion score of top 1,3,10 predicted words
    top1.append(so.index[0] == ending_word)
    top3.append(ending_word in so.index[0:3])
    top10.append(ending_word in so.index[0:10])
    if ending_word in bf.index:
        ranks.append(so.index.get_loc(ending_word) + 1)

In [27]:
# Completion score of top-1 predicted word
sum(top1)/len(top1)

0.4666666666666667

In [28]:
# Completion score of Top-3 predicted word
sum(top3)/len(top3)

0.5333333333333333

In [29]:
# Completion score of Top-10 predicted word
sum(top10)/len(top10)

0.72

In [30]:
# Average entropy
np.mean(entropies)

3.305318748986459

In [31]:
# Entropy deviance
np.std(entropies)

1.873541749007823

In [0]:
# Calculate perplexity which is 2^entropy
perplexities = []
for ent in entropies:
    perplexities.append(2**ent)

In [33]:
# Average perplexity
np.mean(perplexities)

31.409549095646337

In [34]:
# Perplexity deviance
np.std(perplexities)

95.78346306421152

### CONCLUSION/RESULT:

Word prediction algorithm has been implemented using the Stupid Back-off smoothing technique in N-gram Language modelling. The training text was count vectorized into 1-, 2-, 3-, 4- and 5-grams (of which there were many instances, including repeats) and then pruned to keep only those n-grams that appeared more than twice. The test set was count-vectorized only into 5-grams that appeared more than once. Reason being, the final word of a 5-gram that appears more than once in the test set is a bit easier to predict than that of a 5-gram that appears only once. On evaluating the model, we get average entropy of around 3 which gives average perplexity of just 31 i.e on an average, the model was just uncertain among 31 alternative predictions, which is very good for natural-language models.





### CONTRIBUTION:

Personal: 70%, External references: 30%

1. Understanding the concept of N-gram language modelling and its use in word prediction
2. Implementation based on the new smoothing technique called Stupid Backoff introduced by Google in 2007 
3. Incorporated initial text pre-processing that has text cleaning and exploratory data analysis
4. Code based on the Penn Tree Bank dataset used for our word prediction
5. Understanding and implementing the concept of Information Theory; Shannon Entropy and Perplexity for model evaluation

### CITATION/REFERRENCES:

1. https://www.aclweb.org/anthology/D07-1090.pdf
2. https://en.wikipedia.org/wiki/Katz%27s_back-off_model
3. https://rpubs.com/pferriere/dscapreport
4. https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf
5. https://medium.com/@davidmasse8/using-perplexity-to-evaluate-a-word-prediction-model-8820cf3fd3aa
6. https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf
7. https://medium.com/@davidmasse8/predicting-the-next-word-back-off-language-modeling-8db607444ba9
8. https://github.com/b-knight/Text-Prediction-App-with-RShiny-and-Swiftkey-COCA
9. https://medium.com/@datamonsters/text-preprocessing-in-python-steps-tools-and-examples-bf025f872908
10. https://www.kdnuggets.com/2018/03/text-data-preprocessing-walkthrough-python.html

### LICENSES:

**The MIT license:**

https://opensource.org/licenses/MIT  Copyright 2019 Ami Gandhi, Pratik Kadi, Krunal Nanda

**The Creative Commons Attribution 3.0 License:**

<a rel="license" href="http://creativecommons.org/licenses/by/3.0/us/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/3.0/us/80x15.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/3.0/us/">Creative Commons Attribution 3.0 United States License</a>.